<a href="https://colab.research.google.com/github/vydiep/MLProject/blob/main/PT_RNN_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install PySoundFile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import os
import math
import librosa

In [ ]:
DATASET_PATH = "/content/drive/Shareddrives/MLProject/MusicNet/data"
JSON_PATH = "/content/drive/Shareddrives/MLProject/MusicNet/rnn-data.json"
SAMPLE_RATE = 22050 #captures the audio at a certain quality
TRACK_DURATION = 45 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

In [ ]:
"""
Guidance from Resource: 
    https://github.com/musikalkemist/DeepLearningForAudioWithPython/blob/master/12-%20Music%20genre%20classification%3A%20Preparing%20the%20dataset/code/extract_data.py
"""

def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """
    Extracts MFCCs from music data set and saves them into a JSON file
    with composer labels mapped to an integer
    
    Parameters: 
    - dataset_path: path to dataset (str)
    - json_path: path to JSON file for to saving MFCCs (str)
    - num_mfcc: number of coefficients to extract (int)
    - n_fft: interval to apply FFT, measured in number of samples (int)
    - hop_length: sliding window for FFT, measured in number of samples (int)
    - num_segments: number of segments to divide sample tracks into (int)
    """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all sub-folders
    for i, (dir_path, dir_names, file_names) in enumerate(os.walk(dataset_path)):

        # ensure processing sub-folder
        if dir_path is not dataset_path:

            # save composer label (sub-folder name) in the mapping
            label = dir_path.split("/")[-1]
            data["mapping"].append(label)
            print(f"\nProcessing: {label}")

            for file in file_names:

                # process all audio files in composer sub dir
                if file.endswith(".wav"):
                    
                    # load audio file
                    file_path = os.path.join(dir_path, file)
                    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE, duration = TRACK_DURATION)

                    # process all segments of audio file
                    for segment in range(num_segments):

                        # calculate start and finish sample for current segment
                        start = samples_per_segment * segment
                        finish = start + samples_per_segment
                        time_series = signal[start:finish]

                        # extract mfcc
                        mfcc = librosa.feature.mfcc(y=time_series, sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                        mfcc = mfcc.T

                        # only store mfcc features with expected number of vectors
                        if len(mfcc) == num_mfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)
                            print(f"{file_path}, segment:{segment + 1}")

    # save MFCCs in data dictionary to json file
    with open(json_path, "w") as file_ptr:
        json.dump(data, file_ptr, indent=4)

In [ ]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=15)


Processing: Beethoven
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:1
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:2
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:3
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:4
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:5
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:6
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:7
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:8
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:9
/content/drive/Shareddrives/MLProject/MusicNet/data/Beethoven/Beethoven_2382.wav, segment:10
/content/drive/Shareddrives/MLProject/MusicNet/data/Beet